#### Start

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os
# Load token from file
with open('/content/drive/MyDrive/agri-llm/hf_token.txt', 'r') as file:
    hf_token = file.read().strip()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token

In [ ]:
!pip install python-dotenv
!pip install -q -U langchain huggingface_hub sentence_transformers faiss-gpu
!pip install -U langchain-community
!pip install transformers datasets torch scikit-learn numpy

In [8]:
os.chdir('/content/drive/MyDrive/agri-llm')
from models.rag import config_setup as con
from models.rag import fn_rag as rag
from evaluate import fn_ragas as ragas


#### RAGAS Main

In [ ]:
evaluation_questions = {
    "What are the primary uses of maize in industry?":
        "Maize is used for food, animal feed, and as a raw material in various industries like starch and textile.",
    "What is the optimal time for chickpea planting for  peninsular  India?":
        "For  peninsular  India,  first  fort night of  October  is the  best  time  for  chickpea planting."
}

# Initialize your RAG system
rag = rag.RAGSystem(
    documents_path=str(con.Config.DATA_DIR),
)
rag.load_documents()
rag.create_vector_store()
rag.setup_qa_chain()

# Initialize evaluator
evaluator = ragas.RAGEvaluator(rag_system=rag, evaluation_questions=evaluation_questions)

# Run evaluation and print results
eval_results = evaluator.evaluate()
print(ragas.format_evaluation_results(eval_results))